In [1]:
import ee
import geemap
import os
import numpy as np
from matplotlib import pyplot as plt

In [4]:
manaus_shp = '/Users/morgansteckler/Downloads/venus_fgmanaus_footprint/venus_fgmanaus_footprint.shp'
manaus = geemap.shp_to_ee(manaus_shp)

In [47]:
Map = geemap.Map()
Map.centerObject(manaus, 9)
Map

Map(center=[-2.4396454678431994, -59.78996156231432], controls=(WidgetControl(options=['position'], widget=HBo…

In [48]:
#Calculate NDRE and set palette
def addNDRE(image):
    ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE')
    return image.addBands(ndre)
ndre_params = ({'min': 0.0,'max': 1.0, 'palette' : 
               ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']})

#Create cloud mask
cloudBitMask = ee.Number(2).pow(10).int()
cirrusBitMask = ee.Number(2).pow(11).int()
def maskS2clouds(image):
    qa = image.select('QA60')
    # Both flags should be set to zero, indicating clear conditions.
    mask = (qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0)))
    return image.updateMask(mask)

In [49]:
fc= manaus

In [50]:
#Load Sentinel-2 data for 2016 clipped to manaus; select only a few bands
s2 = (ee.ImageCollection('COPERNICUS/S2')
      .filterDate('2016-01-01', '2016-12-31')
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
      .map(lambda image: image.clip(fc))
      .map(maskS2clouds)
      .map(addNDRE)
      .select(['B4', 'B3', 'B2', 'B8', 'B5', 'NDRE']))

#Test out a true color image
single_composite = s2.median()
Map.addLayer(single_composite, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'single composite', False)

In [51]:
#Create months list with 3-month step (end with 4 image collections) w/ median band values
months = ee.List.sequence(1, 12, 3);
def monthStep(m):
    return (s2
            .filter(ee.Filter.calendarRange(start=m,
                                            end=ee.Number(m).add(2),
                                            field='month'))
            .median())

#Apply tri-monthly step
compositesS2 = months.map(monthStep)

In [52]:
#Add bands from previous image to next image in collection
def mergeBands(previous, image):
    return ee.Image(previous).addBands(image)

#Create one composite with band values for each month attached
composite = ee.Image(compositesS2.iterate(mergeBands, ee.Image()));

In [53]:
composite = composite.select(['NDRE', 'NDRE_1', 'NDRE_1_1'])

In [27]:
Map.addLayer(composite, {}, 'complete composite')

In [13]:
bands = composite.bandNames().getInfo()
print(bands)

['NDRE', 'NDRE_1', 'NDRE_1_1']


In [14]:
#K-means
training = composite.sample(region=fc, 
                        scale=20, 
                        numPixels=1000)
clusterer = ee.Clusterer.wekaKMeans(nClusters=4).train(training)
result = composite.cluster(clusterer)
Map.addLayer(result.randomVisualizer(), {})

In [54]:
#Export composite
out_dir = '/Users/morgansteckler/Desktop/All_Months/2016_Composite.tif'
geemap.ee_export_image(composite, out_dir)

Generating URL ...
An error occurred while downloading.
Image.clipToBoundsAndScale: The geometry for image clipping must be bounded.


In [146]:
#Create texture band from NDRE
nir = composite.select('B8')
red = composite.select('B5')
ndre = nir.subtract(red).divide(nir.add(red))
texture = ndre.reduceNeighborhood(reducer=ee.Reducer.stdDev(), kernel=ee.Kernel.square(5)).rename('texture')
Map.addLayer(texture, {'min': 0, 'max': .25}, 'Texture', False)

In [147]:
#Add the texture band
image = composite.addBands(texture)

In [148]:
#Make sure it shows up
bands = image.bandNames().getInfo()
print(bands)

['NDRE', 'B2', 'B3', 'B4', 'B8', 'B5', 'texture']


In [ ]:
#Different Clusterers below:

In [61]:
#Cascade K-means
training = image.sample(region=fc, 
                        scale=20, 
                        numPixels=1000)
clusterer = ee.Clusterer.wekaCascadeKMeans().train(training)
result = image.cluster(clusterer)
Map.addLayer(result.randomVisualizer(), {}, 'Cascade K-means')

In [43]:
#Cobweb
#This didn't work very well...
training = image.sample(region=fc, 
                        scale=20, 
                        numPixels=1000)
clusterer = ee.Clusterer.wekaCobweb().train(training)
result = image.cluster(clusterer)
Map.addLayer(result.randomVisualizer(), {}, 'Cobweb')

In [2]:
#K-means
training = composite.sample(region=fc, 
                        scale=20, 
                        numPixels=1000)
clusterer = ee.Clusterer.wekaKMeans(nClusters=4).train(training)
result = composite.cluster(clusterer)
Map.addLayer(result.randomVisualizer(), {})

In [46]:
#Learning Vector Quantization
#This didn't work very well...
training = image.sample(region=fc, 
                        scale=20, 
                        numPixels=1000)
clusterer = ee.Clusterer.wekaLVQ().train(training)
result = image.cluster(clusterer)
Map.addLayer(result.randomVisualizer(), {}, 'Learning Vector Quantization')

In [47]:
#X-means
training = image.sample(region=fc, 
                        scale=20, 
                        numPixels=1000)
clusterer = ee.Clusterer.wekaXMeans().train(training)
result = image.cluster(clusterer)
Map.addLayer(result.randomVisualizer(), {}, 'X-means')